In [40]:
# SETUP ===================================
import jupyter
import warnings

from src.utils.system import boot, Notify

boot()
warnings.filterwarnings("ignore")



# PACKAGES ================================
import os
import torch
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import gymnasium as gym
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.preprocessing import  RobustScaler

# FRAMEWORK STUFF =========================
from src.config import TOP2_STOCK_BY_SECTOR, FEATURE_COLS
from src.data.feature_pipeline import load_base_dataframe
from src.experiments.experiment_tracker import ExperimentTracker
from src.env.base_timeseries_trading_env import BaseSequenceAwareTradingEnv,SequenceAwareAlphaTradingEnv,SequenceAwareBaselineTradingAgent,SequenceAwareCalmarTradingEnv,SequenceAwareCumulativeTradingEnv,SequenceAwareDrawdownTradingEnv,SequenceAwareHybridTradingEnv,SequenceAwareHybridTradingEnv,SequenceAwareSharpeTradingEnv,SequenceAwareSortinoTradingEnv

from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.policies import ActorCriticPolicy



In [41]:

# ========== CONFIG ==========
EXPERIENCE_NAME = "sequence_awate_agent_design"
RESULTS_PATH = f"data/experiments/{EXPERIENCE_NAME}_barebones_results.csv"
N_EPISODES = 20
N_SEEDS = 3
N_EVAL_EPISODES = 3
AGENT_TYPES = ['mlp', 'lstm', 'transformer_single', 'transformer_multi']
WINDOW_LENGTH = 10  # or any value you want


TRANSACTION_COST = 0

CONFIG = {
    "batch_size": 32,
    "n_steps": 128,
    "total_timesteps": 5000,   
}


"""
features_extractor_kwargs={
    'window_length': WINDOW_LENGTH,
    'n_features': len(FEATURE_COLS),
    'd_model': 32,
    'nhead': ...,
    'num_layers': ...,
}
"""

# --- Load data ---
ohlcv_df = load_base_dataframe()

# --- Experiment tracker ---
experiment_tracker = ExperimentTracker(EXPERIENCE_NAME)



In [42]:
def make_env(df, ticker, feature_cols, episode_length, window_length):
    df_ticker = df[df['symbol'] == ticker].copy()
    return CumulativeTradingEnv(
        df=df_ticker,
        feature_cols=feature_cols,
        episode_length=episode_length,
        transaction_cost=TRANSACTION_COST,
        window_length=window_length,
    )

In [43]:
class TransformerExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, window_length, n_features, d_model=32, nhead=1, num_layers=1):
        super().__init__(observation_space, features_dim=d_model)
        self.window_length = window_length
        self.n_features = n_features
        self.embedding = nn.Linear(n_features, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def forward(self, obs):
        # obs: [batch, window_length * n_features]
        batch = obs.shape[0]
        # reshape flat vector to (batch, window_length, n_features)
        x = obs.view(batch, self.window_length, self.n_features)
        x = self.embedding(x)      # (batch, window_length, d_model)
        x = x.permute(1, 0, 2)    # (window_length, batch, d_model)
        x = self.transformer(x)    # (window_length, batch, d_model)
        # Use last token as pooled output
        return x[-1]              # (batch, d_model)

In [44]:
class TransformerPolicy(ActorCriticPolicy):
    def __init__(self, *args, nhead=1, num_layers=1, window_length=10, n_features=2, **kwargs):
        super().__init__(
            *args,
            features_extractor_class=TransformerExtractor,
            features_extractor_kwargs={
                'window_length': window_length,
                'n_features': n_features,
                'd_model': 32,
                'nhead': nhead,
                'num_layers': num_layers,
            },
            **kwargs
        )

# Unit tests:
1. Output Shapes
2. Window Consistency (Padding at Episode Start)
3. Step Through Environment
4. SB3 Policy Compatibility
5. Transformer Policy Compatibility
6. Action Space and Reward Consistency
7. Episode Generator

In [45]:
# Test 1: Output Shapes

# Test windowed obs shape (flat vs. 2D)
df = ohlcv_df.copy()
feature_cols = FEATURE_COLS
env = BaseSequenceAwareTradingEnv(
    df, feature_cols=feature_cols, episode_length=30, window_length=5, return_sequences=True
)
obs, _ = env.reset()
print("2D window shape:", obs.shape)  # Expect (5, obs_dim)

env_flat = BaseSequenceAwareTradingEnv(
    df, feature_cols=feature_cols, episode_length=30, window_length=5, return_sequences=False
)
obs_flat, _ = env_flat.reset()
print("Flat window shape:", obs_flat.shape)  # Expect (5*obs_dim,)


2D window shape: (5, 25)
Flat window shape: (125,)


In [46]:
# Test 2: Window consistency
env = BaseSequenceAwareTradingEnv(
    df, feature_cols=feature_cols, episode_length=10, window_length=5, return_sequences=True
)
obs, _ = env.reset()
assert np.allclose(obs[0], obs[1]), "Padding at start should repeat first row"
assert obs.shape == (5, len(feature_cols) + len(env.internal_features))
print("Padding and shape OK")


Padding and shape OK


In [47]:
# Test 3: Step Through Environment

env = BaseSequenceAwareTradingEnv(
    df, feature_cols=feature_cols, episode_length=10, window_length=3, return_sequences=True
)
obs, _ = env.reset()
for i in range(8):
    action = env.action_space.sample()
    obs, reward, done, trunc, info = env.step(action)
    print(f"Step {i} | Obs shape: {obs.shape} | Reward: {reward:.5f}")
    if done:
        print("Episode done:", info)
        break

Step 0 | Obs shape: (3, 25) | Reward: 0.00067
Step 1 | Obs shape: (3, 25) | Reward: -0.00168
Step 2 | Obs shape: (3, 25) | Reward: 0.00280
Step 3 | Obs shape: (3, 25) | Reward: -0.00415
Step 4 | Obs shape: (3, 25) | Reward: 0.00380
Step 5 | Obs shape: (3, 25) | Reward: 0.00045
Step 6 | Obs shape: (3, 25) | Reward: 0.00595
Step 7 | Obs shape: (3, 25) | Reward: -0.00463


In [48]:
# SB3 Policy Compatibility
# Train an MLP agent on env with return_sequences=False (flat). 

from stable_baselines3 import PPO

env = BaseSequenceAwareTradingEnv(
    df, feature_cols=feature_cols, episode_length=30, window_length=5, return_sequences=False
)
from stable_baselines3.common.vec_env import DummyVecEnv
vec_env = DummyVecEnv([lambda: env])

model = PPO("MlpPolicy", vec_env, n_steps=8, batch_size=4, verbose=1)
model.learn(total_timesteps=40)
print("SB3 PPO MLP works!")

Using cpu device
---------------------------
| time/              |    |
|    fps             | 88 |
|    iterations      | 1  |
|    time_elapsed    | 0  |
|    total_timesteps | 8  |
---------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 86          |
|    iterations           | 2           |
|    time_elapsed         | 0           |
|    total_timesteps      | 16          |
| train/                  |             |
|    approx_kl            | 0.009850413 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.1        |
|    explained_variance   | -33.8       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00883    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0274     |
|    value_loss           | 0.0939      |
-----------------------------------------
-------------------------------

In [49]:
# Test 5: Transformer Policy Compatibility
# Make sure custom transformer can process the 2D obs by running a forward pass 
# through the extractor to check for shape errors


obs = np.random.randn(2, 5*8).astype(np.float32)  # batch=2, window_length=5, n_features=8
# Extractor expects (batch, window_length*n_features), will reshape internally.
extractor = TransformerExtractor(
    gym.spaces.Box(-np.inf, np.inf, shape=(5*8,), dtype=np.float32), 5, 8
)
with torch.no_grad():
    torch_out = extractor(torch.from_numpy(obs))
print("Transformer output shape:", torch_out.shape)


Transformer output shape: torch.Size([2, 32])


In [50]:
# Test 6: Action Space and Reward Consistency
# mini-episode ti check action output and cumulative reward:

env = BaseSequenceAwareTradingEnv(
    df, feature_cols=feature_cols, episode_length=10, window_length=5, return_sequences=False
)
obs, _ = env.reset()
cumulative = 0
for _ in range(10):
    action = env.action_space.sample()
    obs, reward, done, trunc, info = env.step(action)
    cumulative += reward
    if done:
        print("Episode finished | Cumulative reward:", cumulative)
        print("Info dict:", info)
        break

Episode finished | Cumulative reward: -0.00028617333906170994
Info dict: {'episode_sharpe': 0.07692172235020756, 'episode_sortino': 0.07812661503558227, 'episode_total_reward': -0.00028617333906170994, 'cumulative_return': -4.035402182833714e-05, 'calmar': -0.002803658747317024, 'max_drawdown': 0.014393342936958407, 'win_rate': 0.0, 'alpha': -0.02852024595121727, 'returns': array([-0.        , -0.00168237, -0.00279924,  0.0041545 ,  0.00380186,
        0.00044898,  0.00595171,  0.00463172, -0.01439334]), 'downside': array([-0.00168237, -0.00279924, -0.01439334])}


In [51]:
# Test 7: Episode Generator
# Check that the same seed produces the same episode list across runs.

env = BaseSequenceAwareTradingEnv(df, feature_cols=feature_cols, episode_length=10, window_length=5)
seq1 = env.generate_episode_sequences(train_steps=1000)
env2 = BaseSequenceAwareTradingEnv(df, feature_cols=feature_cols, episode_length=10, window_length=5)
seq2 = env2.generate_episode_sequences(train_steps=1000)
assert seq1 == seq2, "Episode sequences should be the same for same seed!"
print("Episode generator determinism OK")

Episode generator determinism OK
